In [ ]:
!pip install -q dwave-ocean-sdk
!pip install -q ortools

# Problem Statement
A person named John wants to find best investment options, he has total budget 100,000 USD.
There are 3 funds,
* A) Fund A is low risk fund with historical gain of 5%
* B) Fund B is medium risk fund with historical gain of 10%
* c) Fund C is High risk fund with historical gain of 12%

He wish to control the risk of his investment with maximum of 10% of his money in the investment with high risk fund and 20% with medium risk fund.
***Maximize the return***

**Mathematical Formulation**
  **Variable**
*    R1, R2, R3 are the return of investment A, B, C
*    C1, C2, C3 are the investment capital in fund A, B, C

  **Constraints**
*    C1+C2+C3 = 100,000
*    R1 = 0.05C1
*    R2 = 0.10C2
*    R3 = 0.12C3
*    0<=C2<= 0.2*100,000
*    0<=C3<= 0.1*100,000

  **Objective Function**
*    maximize R1+ R2 +R3

## Classical Solution using GLOP

In [57]:
from ortools.linear_solver import pywraplp

# Create the linear solver with the GLOP backend made by Google.
solver = pywraplp.Solver.CreateSolver('GLOP')

total_budget = 100000
# Adding variables
# capital of investment
C1 = solver.IntVar(0, total_budget, 'C1')
C2 = solver.IntVar(0, total_budget, 'C2')
C3 = solver.IntVar(0, total_budget, 'C3')

# return of investment
R1 = solver.IntVar(0, total_budget, 'R1')
R2 = solver.IntVar(0, total_budget, 'R2')
R3 = solver.IntVar(0, total_budget, 'R3')

# Defining the expressions for return of investment
solver.Add(R1 == 0.05 * C1)  # Low risk fund has 5% gain historical per year
solver.Add(R2 == 0.10 * C2)  # Medium Risk Fund has 10% gain per year
solver.Add(R3 == 0.12 * C3)  # High risk fund has 12% gain per year

# Total weighted return
total_return = R1 + R2 + R3

# Adding constraint: total allocation equals total budget
solver.Add(C1 + C2 + C3 == total_budget)

# Adding constraint: minimum allocation percentage for C2 and C3
min_allocation_percentage = 0.05 * total_budget  # 5% of total budget
solver.Add(C2 >= min_allocation_percentage)
solver.Add(C3 >= min_allocation_percentage)

# Adding constraint: maximum allocation percentage for C2 and C3
max_allocation_percentage_C2 = 0.2 * total_budget  # 20% of total budget
max_allocation_percentage_C3 = 0.1 * total_budget  # 10% of total budget
solver.Add(C2 <= max_allocation_percentage_C2)
solver.Add(C3 <= max_allocation_percentage_C3)

# Maximize total return
solver.Maximize(total_return)

# Solve the problem
result = solver.Solve()

# Check if it is optimal
if result == pywraplp.Solver.OPTIMAL:
    print('Optimal Found')

# Print the solution
print('C1:', C1.solution_value())
print('C2:', C2.solution_value())
print('C3:', C3.solution_value())
print('R1:', R1.solution_value())
print('R2:', R2.solution_value())
print('R3:', R3.solution_value())
# Calculate return percentages
return_percentage_C1 = (R1.solution_value() / C1.solution_value()) * 100
return_percentage_C2 = (R2.solution_value() / C2.solution_value()) * 100
return_percentage_C3 = (R3.solution_value() / C3.solution_value()) * 100
total_return = R1.solution_value()+R2.solution_value()+R3.solution_value()
print("Total return",total_return )
print('Return Percentage for C1:', return_percentage_C1, '%')
print('Return Percentage for C2:', return_percentage_C2, '%')
print('Return Percentage for C3:', return_percentage_C3, '%')

Optimal Found
C1: 70000.0
C2: 20000.0
C3: 10000.0
R1: 3500.0
R2: 2000.0
R3: 1200.0
Total return 6700.0
Return Percentage for C1: 5.0 %
Return Percentage for C2: 10.0 %
Return Percentage for C3: 12.0 %


In [60]:
import dimod
from dwave.system import LeapHybridCQMSampler

# Define the total budget
total_budget = 100000

# Define the decision variables as binary variables
C1_qc = dimod.Integer('C1_qc')
C2_qc = dimod.Integer('C2_qc')
C3_qc = dimod.Integer('C3_qc')
R1_qc = dimod.Integer('R1_qc')
R2_qc = dimod.Integer('R2_qc')
R3_qc = dimod.Integer('R3_qc')

# Total weighted return
total_return_qc = R1_qc + R2_qc + R3_qc

# Create a ConstrainedQuadraticModel
cqm = dimod.ConstrainedQuadraticModel()

# Adding objective function to be minimized or maximized
cqm.set_objective(total_return_qc)

# Adding constraints to the model
# Defining the expressions for return of investment
cqm.add_constraint((R1_qc - 0.05 * C1_qc) == 0, label="low risk constraint")  # Low risk fund has 5% gain historical per year
cqm.add_constraint((R2_qc - 0.10 * C2_qc) == 0, label="medium risk constraint")  # Medium Risk Fund has 10% gain per year
cqm.add_constraint((R3_qc - 0.12 * C3_qc) == 0, label="high risk constraint")  # High risk fund has 12% gain per year


# Adding constraint: total allocation equals total budget
cqm.add_constraint(C1_qc + C2_qc + C3_qc == total_budget)

# Adding constraint: minimum allocation percentage for C2 and C3
min_allocation_percentage = 0.05 * total_budget  # 5% of total budget
cqm.add_constraint(C2_qc >= min_allocation_percentage,label="C2 min allocation constraint")
cqm.add_constraint(C3_qc >= min_allocation_percentage,label="C3 min allocation constraint")

# Adding constraint: maximum allocation percentage for C2 and C3
max_allocation_percentage_C2 = 0.2 * total_budget  # 20% of total budget
max_allocation_percentage_C3 = 0.1 * total_budget  # 10% of total budget
cqm.add_constraint(C2_qc <= max_allocation_percentage_C2,label="C2 max allocation constraint")
cqm.add_constraint(C3_qc <= max_allocation_percentage_C3,label="C3 max allocation constraint")


# Create an instance of the LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token='DEV-081b757ac8fa9601e4fd99a1710375235deaa0c4')

# Sample from the CQM
sample = sampler.sample_cqm(cqm, label="Portfolio Optimization")

# Extract the optimal solution
energies = sample.data_vectors['energy']
optimal_energy = max(energies)
optimal_samples = [sample for sample, energy in zip(sample.samples(), energies) if energy == optimal_energy]
optimal_solution = optimal_samples[0]
print(optimal_solution)

# Print the solution
print('C1:', optimal_solution['C1_qc'])
print('C2:', optimal_solution['C2_qc'])
print('C3:', optimal_solution['C3_qc'])
print('R1:', optimal_solution['R1_qc'])
print('R2:', optimal_solution['R2_qc'])
print('R3:', optimal_solution['R3_qc'])


# Calculate return percentages
return_percentage_C1_qc = (optimal_solution['R1_qc'] / optimal_solution['C1_qc']) * 100
return_percentage_C2_qc = (optimal_solution['R2_qc'] / optimal_solution['C2_qc']) * 100
return_percentage_C3_qc = (optimal_solution['R3_qc'] / optimal_solution['C3_qc']) * 100
total_return_qc = optimal_solution['R1_qc'] + optimal_solution['R2_qc'] + optimal_solution['R3_qc']

print("Total return:", total_return_qc)
print('Return Percentage for C1:', return_percentage_C1_qc, '%')
print('Return Percentage for C2:', return_percentage_C2_qc, '%')
print('Return Percentage for C3:', return_percentage_C3_qc, '%')

{'C1_qc': 71638.0, 'C2_qc': 18532.0, 'C3_qc': 9833.0, 'R1_qc': 3582.0, 'R2_qc': 1854.0, 'R3_qc': 1180.0}
C1: 71638.0
C2: 18532.0
C3: 9833.0
R1: 3582.0
R2: 1854.0
R3: 1180.0
Total return: 6616.0
Return Percentage for C1: 5.000139590720009 %
Return Percentage for C2: 10.004316857327865 %
Return Percentage for C3: 12.000406793450626 %


## Comparison

In [61]:
# Print the solution
print("=====================Classical Solution ============================")
print('C1:', C1.solution_value())
print('C2:', C2.solution_value())
print('C3:', C3.solution_value())
print('R1:', R1.solution_value())
print('R2:', R2.solution_value())
print('R3:', R3.solution_value())
# Calculate return percentages
return_percentage_C1 = (R1.solution_value() / C1.solution_value()) * 100
return_percentage_C2 = (R2.solution_value() / C2.solution_value()) * 100
return_percentage_C3 = (R3.solution_value() / C3.solution_value()) * 100
total_return = R1.solution_value()+R2.solution_value()+R3.solution_value()
print("Total return",total_return )
print('Return Percentage for C1:', return_percentage_C1, '%')
print('Return Percentage for C2:', return_percentage_C2, '%')
print('Return Percentage for C3:', return_percentage_C3, '%')


# Print the solution
print("=====================Quantum Solution ============================")
# Print the solution
print('C1:', optimal_solution['C1_qc'])
print('C2:', optimal_solution['C2_qc'])
print('C3:', optimal_solution['C3_qc'])
print('R1:', optimal_solution['R1_qc'])
print('R2:', optimal_solution['R2_qc'])
print('R3:', optimal_solution['R3_qc'])

# Calculate return percentages
return_percentage_C1_qc = (optimal_solution['R1_qc'] / optimal_solution['C1_qc']) * 100
return_percentage_C2_qc = (optimal_solution['R2_qc'] / optimal_solution['C2_qc']) * 100
return_percentage_C3_qc = (optimal_solution['R3_qc'] / optimal_solution['C3_qc']) * 100
total_return_qc = optimal_solution['R1_qc'] + optimal_solution['R2_qc'] + optimal_solution['R3_qc']

print("Total return:", total_return_qc)
print('Return Percentage for C1:', return_percentage_C1_qc, '%')
print('Return Percentage for C2:', return_percentage_C2_qc, '%')
print('Return Percentage for C3:', return_percentage_C3_qc, '%')

print("Return Difference QC - Classical",total_return_qc - total_return)


=====================Classical Solution ============================
C1: 70000.0
C2: 20000.0
C3: 10000.0
R1: 3500.0
R2: 2000.0
R3: 1200.0
Total return 6700.0
Return Percentage for C1: 5.0 %
Return Percentage for C2: 10.0 %
Return Percentage for C3: 12.0 %
=====================Quantum Solution ============================
C1: 71638.0
C2: 18532.0
C3: 9833.0
R1: 3582.0
R2: 1854.0
R3: 1180.0
Total return: 6616.0
Return Percentage for C1: 5.000139590720009 %
Return Percentage for C2: 10.004316857327865 %
Return Percentage for C3: 12.000406793450626 %
Return Difference QC - Classical -84.0
